<a href="https://colab.research.google.com/github/WeustiS/ProjectX/blob/main/Data_Parsing_Demand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [157]:
import requests as r
from io import BytesIO, StringIO
import zipfile
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from google.colab import drive
import time
import pickle

In [158]:
base_url = r'http://oasis.caiso.com/oasisapi/SingleZip?'
# ?queryname=PRC_AS&startdatetime=20191018T07:00-0000&enddatetime=20191019T07:00-0000
# &market_run_id=DAM&version=1&as_type=ALL&as_region=ALL

def get_data(params):
    # building URL
    url = base_url
    for key, value in params.items():
        url += f"{key}={value}&"
    data = r.get(url[:-1]).content
    print("URL:", url)
    try:
        filecontent = BytesIO(data)
    except TypeError:
        filecontent = StringIO(data)

    try:
            # have zipfile
        z = zipfile.ZipFile(filecontent)
    except zipfile.BadZipfile:
        return None

        # have unzipped content
    unzipped = [z.read(thisfile) for thisfile in z.namelist()]
    z.close()

        # return
    return unzipped

In [159]:
# function to convert XML to dataframe for nodes
def parse_XML(root):
  df = pd.DataFrame(columns=['Variable', 'Node', 'Date','Interval Number','Value'])
  j = 0
  #
  for i in root[1][0][4][1:]:
    #print(i[2].text, float(i[6].text))\
    new_row = {'Variable':i[0].text, 'Node':i[1].text, 'Date':i[2].text, 'Interval Number':float(i[3].text),'Value':float(i[6].text)}
    #append row to the dataframe
    df = df.append(new_row, ignore_index=True)
  return df

In [160]:
# # function to convert XML to 
# def parse_XML_energy(root):
#   df = pd.DataFrame(columns=['Variable', 'Date','Interval Number','Value', 'Trading Hub', 'Renewable Energy Type'])
#   for j in range(1,7):
#     for i in root[1][0][j][1:]:
#       print(i[2].text, float(i[5].text))
#       #new_row = {'Variable':i[0].text, 'Date':i[1].text, 'Interval Number':int(i[2].text), 'Value':float(i[5].text),'Trading Hub':i[6].text, 'Renewable Energy Type':i[7].text}
#     #append row to the dataframe0
#     #df = df.append(new_row, ignore_index=True)
#  # return df

In [161]:

#trial cell
#payload = {
#            'queryname': 'PRC_INTVL_LMP', 
#            'startdatetime': '20200102T08:00-0000',
#            'enddatetime':'20200103T08:00-0000',
#            'version': '1',
#            'market_run_id': 'RTM',
#            'node': 'PERRY_6_N001'
#          }
##xml = get_data(payload)
#xml_to_string = str(xml[0],'utf-8')
#display(xml_to_string)
#root = ET.fromstring(xml_to_string)
#data = parse_XML(root)
##DF_list.append(data)

In [162]:
# function to convert XML to 
def parse_XML_energy(root):
  df = pd.DataFrame(columns=['Variable', 'Date','Interval Number','Value', 'Trading Hub', 'Renewable Energy Type'])
  for j in range(1,7):
    for i in root[1][0][j][1:]:
      #print(i[2].text, float(i[6].text))\
      new_row = {'Variable':i[0].text, 'Date':i[1].text, 'Interval Number':i[2].text, 'Value':float(i[5].text),'Trading Hub':i[6].text, 'Renewable Energy Type':i[7].text}
      df = df.append(new_row, ignore_index=True)
    #append row to the dataframe0
  return df

In [163]:
def parse_XML_demand(root):
  df = pd.DataFrame(columns=['Variable', 'Resource Name', 'Date', 'Interval Number', 'Value'])
  count = 0
  for i in root[1][0]:
    count = count +1
  for i in range(1, count):
    for j in root[1][0][i][1:]:
      #print(j[0].text, j[1].text, j[2].text,j[3].text, j[6].text)
      new_row = {'Variable':j[0].text, 'Resource Name':j[1].text, 'Date':j[2].text, 'Interval Number':j[3].text, 'Value':float(j[6].text)}
      df = df.append(new_row, ignore_index=True)
  return df

In [164]:
##trial cell
payload = {
           'queryname': 'SLD_FCST', 
           'startdatetime': '20200101T08:00-0000',
           'enddatetime':'20200102T08:00-0000',
           'version': '1',
           'market_run_id': 'ACTUAL',
           'execution_type':'ACTUAL'
         }
xml = get_data(payload)
xml_to_string = str(xml[0],'utf-8')
#display(xml_to_string)
root = ET.fromstring(xml_to_string)
trial_df = parse_XML_demand(root)
trial_df.head()
##DF_list.append(data)

URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_FCST&startdatetime=20200101T08:00-0000&enddatetime=20200102T08:00-0000&version=1&market_run_id=ACTUAL&execution_type=ACTUAL&


,Variable,Resource Name,Date,Interval Number,Value
0,SYS_FCST_ACT_MW,AZPS,2020-01-01,1,3010.0
1,SYS_FCST_ACT_MW,AZPS,2020-01-01,18,3280.0
2,SYS_FCST_ACT_MW,AZPS,2020-01-01,17,3091.0
3,SYS_FCST_ACT_MW,AZPS,2020-01-01,2,3000.0
4,SYS_FCST_ACT_MW,AZPS,2020-01-01,6,3354.0


In [165]:
DF_list= list()
payload = {
           'queryname': 'SLD_REN_FCST', 
           'startdatetime': '20200102T08:00-0000',
           'enddatetime':'20200103T08:00-0000',
           'version': '1',
           'market_run_id': 'RTD',
           #'node': 'PERRY_6_N001'
         }
xml = get_data(payload)
xml_to_string = str(xml[0],'utf-8')
#display(xml_to_string)
root = ET.fromstring(xml_to_string)
data = parse_XML_energy(root)
DF_list.append(data)

URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_REN_FCST&startdatetime=20200102T08:00-0000&enddatetime=20200103T08:00-0000&version=1&market_run_id=RTD&


In [166]:
market_type = {'RTD', 'ACTUAL','DAM','HASP','RTPD'}

In [167]:
nodes_list_1 = ['WOHLER_6_N001', 'SOLEDAD_6_N001', 'ZAMORA_1_N001', 'SILVERDO_1_N002', 'ZANKER_1_N003']
nodes_list_2 = ['WINTERS_6_N001', 'REDBUD_1_N004']
nodes_list_3 = ['PERRY_6_N001', 'SANDBAR_7_B1', 'TULUCAY_6_N001']
nodes_list_4 = ['WOODSIDE_6_N004', 'WOODACRE_6_N001', 'ROBROY_1_N001', 'WATRSHED_6_N001','OLEMA_6_N001']
nodes_list_5 = ['BOLINAS_6_N001', 'TOCALOMA_6_N001','STONYPT_1_N001','STAFFORD_6_N101','WOLFSKIL_7_B1']
nodes_list_6 = ['VICTOR_6_N002', 'SPRINGVL_6_N006', 'PHILO_6_N001','MALCHA_7_B1', 'EASTWOOD_7_B1', 'CONTROLX_1_N010', 'WAHTOKE_1_N002', 'RTRACK_1_N001', 'QUEBEC_1_N001', 'WHEATLND_6_N006']
nodes_list_8 = ['MUSLSLGH_6_N001', 'S0479_7_N003', 'WEMRSWS_6_N001', 'SHNGLESR_1_N103','JOLON_6_N001']
nodes_list_11 = ['STMARIA_1_N101', 'OCEANO_1_N004','GRANDIS_1_N009', 'INDIANV_7_B1','WHEELBR1_7_B1']


In [ ]:
market_type_demand = ['RTM','ACTUAL', '2DA', '7DA', 'DAM']

In [ ]:
DF_list= list()
for market in market_type_demand:
  time.sleep(2)
  for month in range(1, 13):
    time.sleep(2)
    for i in range(1,31):
      if (month == 2 and i > 27):
        continue
      if ((month == 4 or month == 6 or month == 9 or month == 11) and i > 29):
        continue
      time.sleep(5)
      if (month < 10):
        if (i < 10):
          # changing the start and end date for scraping purposes
          start_date = '20180' + str(month)+'0' + str(i)+'T08:00-0000'
          end_date = '20180'+str(month) + '0'+str(i+1)+'T08:00-0000'
          # dynamically changing payload
          if (market != 'RTM'):
            payload = {
            'queryname': 'SLD_FCST', 
            'startdatetime': start_date,
            'enddatetime': end_date,
            'version': '1',
            'market_run_id': market,
            'execution_type': market
            #'node': node
            }
          else:
            payload = {
            'queryname': 'SLD_FCST', 
            'startdatetime': start_date,
            'enddatetime': end_date,
            'version': '1',
            'market_run_id': market,
            'execution_type': 'RTD'
            #'node': node
            }
          xml = get_data(payload)
          if (xml is None):
            continue
          # converting xml to dataframe
          xml_to_string = str(xml[0],'utf-8')
          root = ET.fromstring(xml_to_string)
          data = parse_XML_demand(root)
          DF_list.append(data)
        else:
          start_date = '20180' + str(month)+ str(i)+'T08:00-0000'
          end_date = '20180'+str(month) + str(i+1)+'T08:00-0000'
          if (market != 'RTM'):
            payload = {
            'queryname': 'SLD_FCST', 
            'startdatetime': start_date,
            'enddatetime': end_date,
            'version': '1',
            'market_run_id': market,
            'execution_type': market
            #'node': node
            }
          else:
            payload = {
            'queryname': 'SLD_FCST', 
            'startdatetime': start_date,
            'enddatetime': end_date,
            'version': '1',
            'market_run_id': market,
            'execution_type': 'RTD'
            #'node': node
            }
          xml = get_data(payload)
          if (xml is None):
            continue
          xml_to_string = str(xml[0],'utf-8')
          root = ET.fromstring(xml_to_string)
          data = parse_XML_demand(root)
          DF_list.append(data)
      else:
        if (i < 10):
          # changing the start and end date for scraping purposes
          start_date = '2018' + str(month)+ '0'+ str(i)+'T08:00-0000'
          end_date = '2018'+str(month)+'0'+str(i+1)+'T08:00-0000'
          # dynamically changing payload
          if (market != 'RTM'):
            payload = {
            'queryname': 'SLD_FCST', 
            'startdatetime': start_date,
            'enddatetime': end_date,
            'version': '1',
            'market_run_id': market,
            'execution_type': market
            #'node': node
            }
          else:
            payload = {
            'queryname': 'SLD_FCST', 
            'startdatetime': start_date,
            'enddatetime': end_date,
            'version': '1',
            'market_run_id': market,
            'execution_type': 'RTD'
            #'node': node
            }
          xml = get_data(payload)
          if (xml is None):
            continue
          # converting xml to dataframe
          xml_to_string = str(xml[0],'utf-8')
          root = ET.fromstring(xml_to_string)
          data = parse_XML_demand(root)
          DF_list.append(data)
        else:
          start_date = '2018' + str(month)+ str(i)+'T08:00-0000'
          end_date = '2018' + str(month)+str(i+1)+'T08:00-0000'
          if (market!= 'RTM'):
            payload = {
            'queryname': 'SLD_FCST', 
            'startdatetime': start_date,
            'enddatetime': end_date,
            'version': '1',
            'market_run_id': market,
            'execution_type': market
            #'node': node
            }
          else:
            payload = {
            'queryname': 'SLD_FCST', 
            'startdatetime': start_date,
            'enddatetime': end_date,
            'version': '1',
            'market_run_id': market,
            'execution_type': 'RTD'
            #'node': node
            }
          xml = get_data(payload)
          if (xml is None):
            continue
          xml_to_string = str(xml[0],'utf-8')
          root = ET.fromstring(xml_to_string)
          data = parse_XML_demand(root)
          DF_list.append(data)

URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_FCST&startdatetime=20190101T08:00-0000&enddatetime=20190102T08:00-0000&version=1&market_run_id=RTM&execution_type=RTD&
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_FCST&startdatetime=20190102T08:00-0000&enddatetime=20190103T08:00-0000&version=1&market_run_id=RTM&execution_type=RTD&
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_FCST&startdatetime=20190103T08:00-0000&enddatetime=20190104T08:00-0000&version=1&market_run_id=RTM&execution_type=RTD&
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_FCST&startdatetime=20190104T08:00-0000&enddatetime=20190105T08:00-0000&version=1&market_run_id=RTM&execution_type=RTD&
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_FCST&startdatetime=20190105T08:00-0000&enddatetime=20190106T08:00-0000&version=1&market_run_id=RTM&execution_type=RTD&
URL: http://oasis.caiso.com/oasisapi/SingleZip?queryname=SLD_FCST&startdatetime=20190106T08:00-0000&

In [ ]:
DF_list
df = pd.concat(DF_list)
df

,Variable,Resource Name,Date,Interval Number,Value
0,SYS_FCST_5MIN_MW,AZPS,2019-01-01,201,3450.91
1,SYS_FCST_5MIN_MW,AZPS,2019-01-01,205,3639.32
2,SYS_FCST_5MIN_MW,AZPS,2019-01-01,206,3596.66
3,SYS_FCST_5MIN_MW,AZPS,2019-01-01,208,3640.11
4,SYS_FCST_5MIN_MW,AZPS,2019-01-01,209,3671.20
...,...,...,...,...,...
331,SYS_FCST_DA_MW,VEA-TAC,2019-12-30,19,93.84
332,SYS_FCST_DA_MW,VEA-TAC,2019-12-30,5,92.67
333,SYS_FCST_DA_MW,VEA-TAC,2019-12-30,23,86.72
334,SYS_FCST_DA_MW,VEA-TAC,2019-12-30,13,82.12


In [ ]:
pickle.dump(df, open("2018_demand", "wb"))

In [156]:
# code to save data in google drive directly
#data = parse_XML(root)
drive.mount('drive', force_remount=True)
df.to_csv('2018_demand.csv', index = False)
!cp 2018_demand.csv "drive/My Drive/ProjectX Data"

Mounted at drive


In [ ]:
00

0